# 🚔 DispatchAI: Remote Brain (Colab Server)
**Run this notebook on a T4 GPU Runtime.**
This server acts as the AI processing hub for the EAEDS Control system.

**Stack:**
*   **STT:** `faster-whisper-large-v3`
*   **Emotion:** `opensmile` (Acoustic) + `BERT` (Semantic)
*   **LLM:** `spikecodes/ai-911-operator` (Mistral-7B Adapter)
*   **Tunnel:** `cloudflared` (Reliable & Free)

In [1]:
# 1. Install Dependencies (Approx 3-5 mins)
!sudo apt-get install -y ffmpeg pciutils
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/peft
!pip install accelerate bitsandbytes faster-whisper opensmile fastapi uvicorn python-multipart pydub nest_asyncio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (289 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <>

In [2]:
# 2. Load Models (The Brain Cells)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from faster_whisper import WhisperModel
import opensmile
import os

print("⏳ Loading Whisper STT...")
stt_model = WhisperModel("large-v3", device="cuda", compute_type="float16")

print("⏳ Loading OpenSMILE...")
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
)

print("⏳ Loading Mistral 7B + 911 Adapter...")
# Base Model
base_model_id = "mistralai/Mistral-7B-v0.1"
adapter_id = "spikecodes/ai-911-operator"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

print("✅ All Systems Online. GPU Ready.")

⏳ Loading Whisper STT...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


⏳ Loading OpenSMILE...
⏳ Loading Mistral 7B + 911 Adapter...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

✅ All Systems Online. GPU Ready.


In [ ]:
# 3. The Server Logic (With Cloudflare Tunnel)
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import uvicorn
import nest_asyncio
import shutil
import json
import time
import subprocess
import re
from threading import Thread

app = FastAPI()

# Add CORS Middleware to allow requests from localhost:3000
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

nest_asyncio.apply()

@app.post("/process_emergency")
async def process_emergency(audio: UploadFile = File(...)):
    start_time = time.time()

    # A. Save Audio Raw
    raw_filename = f"raw_{audio.filename}"
    clean_filename = "clean_input.wav"

    with open(raw_filename, "wb") as buffer:
        shutil.copyfileobj(audio.file, buffer)

    # A2. CONVERT TO WAV (Fixes Browser WebM/Opus issue)
    # Browsers send WebM but name it .wav. OpenSMILE hates this. ffmpeg fixes it.
    subprocess.run(["ffmpeg", "-y", "-i", raw_filename, "-ar", "16000", "-ac", "1", "-c:a", "pcm_s16le", clean_filename], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # B. Transcribe (Whisper)
    print("🔊 Transcribing...")
    # Use the CLEAN file now
    segments, info = stt_model.transcribe(clean_filename, beam_size=5)
    transcript = " ".join([segment.text for segment in segments])
    print(f"🗣️ Transcript: {transcript}")

    # C. Acoustic Analysis (OpenSMILE)
    try:
        features = smile.process_file(clean_filename)
        pitch_mean = features['F0semitoneFrom27.5Hz_sma3nz_amean'].values[0]
        loudness_mean = features['loudness_sma3_amean'].values[0]
        emotion_tag = "Calm"
        if pitch_mean > 35 or loudness_mean > 2.0:
            emotion_tag = "Panic"
        print(f"💓 Acoustic Emotion: {emotion_tag}")
    except Exception as e:
        print(f"⚠️ Emotion Analysis Failed: {e}")
        emotion_tag = "Neutral"

    # D. LLM Response
    print("🧠 Thinking...")
    prompt = f"911 Operator: 9-1-1, what's your emergency?\nCaller: {transcript}\n911 Operator:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True).split("911 Operator:")[-1].strip()

    print(f"🤖 Response: {response_text}")

    return JSONResponse(content={
        "transcript": transcript,
        "response": response_text,
        "acoustic_emotion": emotion_tag,
        "processing_time": time.time() - start_time
    })

print("🚀 Starting Cloudflare Tunnel...")
def start_tunnel():
    # Download and run cloudflared
    subprocess.run(["wget", "-q", "-nc", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"])
    subprocess.run(["chmod", "+x", "cloudflared-linux-amd64"])

    # Run tunnel and write output to a file so we can read it
    with open("cf_logs.txt", "w") as log_file:
        subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8000"], stderr=log_file, stdout=log_file)

    # Monitor log file for URL
    found = False
    for _ in range(50):
        time.sleep(2)
        try:
            with open("cf_logs.txt", "r") as f:
                content = f.read()
                matches = re.findall(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
                for url in matches:
                    print(f"\n🔗 CONNECT TO THIS URL: {url}\n")
                    found = True
                    return
        except Exception as e:
            pass
    if not found:
        print("❌ Tunnel failed to start. Check cf_logs.txt")

Thread(target=start_tunnel, daemon=True).start()
time.sleep(3)

print("🚀 Starting Server...")
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()

🚀 Starting Cloudflare Tunnel...
🚀 Starting Server...


INFO:     Started server process [623]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



🔗 CONNECT TO THIS URL: https://brighton-hang-initially-reconstruction.trycloudflare.com

INFO:     175.107.216.135:0 - "GET /docs HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  ہیلپ فائر
💓 Acoustic Emotion: Panic
🧠 Thinking...
🤖 Response: 9-1-1, what's your emergency?
Caller
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  I am in fire, my clothes are burning
💓 Acoustic Emotion: Calm
🧠 Thinking...
🤖 Response: What house?
Caller
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  I am at Golden Gate
💓 Acoustic Emotion: Calm
🧠 Thinking...
🤖 Response: What'
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  I am in the house
💓 Acoustic Emotion: Calm
🧠 Thinking...
🤖 Response: What's
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript: 
💓 Acoustic Emotion: Calm
🧠 Thinking